# Préliminaires

**ATTENTION !!!! Etape 0 :**

Enregistrer une copie de ce notebook sur VOTRE drive et la renommer de la manière suivante :


> IAresolution**VOTRENOM**.ipynb

Vous travaillerez ensuite UNIQUEMENT sur cette copie.


---

**Fin de l'étape 0 : installer les fichiers nécessaires en exécutant le bloc de code suivant.**


In [ ]:
!pip install python-sat
!pip install python-constraint
!wget https://gitlab.irit.fr/ia-resolutionpb/projetiaresolutionpb/-/raw/main/projet.zip
!unzip projet.zip
!wget https://gitlab.irit.fr/ia-resolutionpb/projetiaresolutionpb/-/raw/main/Data.zip
!unzip Data.zip
!wget https://gitlab.irit.fr/ia-resolutionpb/projetiaresolutionpb/-/raw/main/Doc.zip
!unzip Doc.zip

La documentation sur les fichiers du projet se trouve dans le fichier *Doc.zip* que vous pouvez télécharger depuis la liste des fichiers et installer en le décompressant sur votre machine.

Pour chaque étape du projet, les fichiers de code du projet que vous allez devoir utiliser ou modifier ont déjà été placés dans des blocs de code. Vous pouvez donc faire les modifications demandées et directement tester votre code sur la plateforme GoogleCoLab.


---


**A la fin du projet, vous devrez télécharger votre notebook (donc le fichier *IAresolutionVOTRENOM.ipynb*) et le déposer sous moodle.**

# Contexte du travail à faire

**Rappel du Contexte**

R2D2 est un robot placé dans un monde 2D représenté par un graphe non orienté : les arêtes représentent les routes que R2D2 peut suivre, alors que les sommets représentent les lieux où R2D2 a des choses à faire. Les arêtes seront pondérées pour représenter la longueur du chemin que doit parcourir R2D2 pour aller du sommet origine de l’arête au sommet arrivée de l’arête. Et chaque sommet est pondéré par sa position dans le plan 2D du monde (coordonnées euclidiennes).

On prendra comme hypothèse que R2D2 connaı̂t le monde dans lequel il est placé.
Le travail que doit faire R2D2 : déposer 1 cube de couleur à chaque lieu de manière à ce qu’il y ait dans deux lieux voisins (c-à-d liés par une arête) des cubes de couleur différente. On considérera que R2D2 dispose de suffisamment de cubes.

Le travail de R2D2 va se décomposer en plusieurs tâches :

*   tâche 1 : Au début, R2D2 décide de n’utiliser que 3 couleurs. Il va donc devoir “raisonner” pour savoir si ces 3 couleurs lui suffisent ou pas pour réaliser son travail.
*   tâche 2 : Ensuite, R2D2 cherche à savoir comment il va aller déposer les cubes le plus rapidement possible. Pour cela, il cherche des chemins les plus courts en terme de distance parcourue.
*   tâche 3 : Finalement, R2D2 cherche à savoir combien il lui faut de couleurs a minima pour réaliser son travail.



# Etape 1

**Etape 1 = TP 1 et 2** Réalisation de la tâche 1 :

Vous devrez identifier le nom et le type du problème, proposer un encodage en logique propositionnelle et le résoudre en utilisant le solver SAT fourni.
Pour cela vous compléterez la classe *Etape1* du package *etape1* et vous l’exécuterez. Il peut être souhaitable de compléter les tests déjà proposés.

In [75]:
"""
module pour l'etape 1
"""
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.solvers.SolverSAT import SolverSAT


class Etape1 :
    """
    classe pour realiser l'etape 1 du projet (tache 1)
    """
    # attributs
    # //////////////////////////////////////////////
    g : GrapheDeLieux
    """
    le graphe representant le monde
    """
    base : list
    """
    la base de clauses representant le probleme.
    C'est une liste de listes d'entiers, un entier par variable,
    (positif si literal positif, negatif sinon).
    Attention le 0 n'est pas autorise pour represente une variable
    maj par majBase
    """
    nbVariables : int
    """
    le nb de variables utilisees pour representer le probleme
    maj par majBase
    """

    # constructeur
    # //////////////////////////////////////////////
    def __init__ (self, fn : str, form : bool) :
        """
        constructeur

        :param fn: le nom du fichier dans lequel sont donnes les sommets et les aretes

        :param form: permet de distinguer entre les differents types de fichier
         (pour ceux contenant des poids et des coordonnees, form est True ;
          pour les autres, form est False)
        """
        self.g = GrapheDeLieux.loadGraph(fn,form)
        self.base = []
        self.nbVariables = 0

    # methodes
    # //////////////////////////////////////////////
    def majBase(self, x : int) :
        """
        methode de maj de la base de clauses et du nb de variables en fonction du pb traite

        :param x: parametre servant pour definir la base qui representera le probleme
        """
        # A ECRIRE par les etudiants en utilisant le contenu de g
        # ajout possible de parametre => modifier aussi l'appel ds le main
        sommets = self.g.getNbSommets()

        couleurs=x
        #print(couleurs)
        teta=[]

        #print(sommets)
        #print(couleurs)
        formule=[]

        #print(range(sommets))
        for sommet in range(1,sommets+1):
            for couleur in range(1,couleurs+1):
                formule.append(couleur+sommet*1000)
                for i in range(1,couleurs):
                    if(couleur != i+1 and couleur < i+1):
                        teta.append([-(couleur+sommet*1000),-(i+1+sommet*1000)])
            teta.append(formule[:])
            formule.clear()
            adj=self.g.getAdjacents(sommet-1)
            for cote in adj:
                phi=[]
                for couleur in range(1,couleurs+1):
                    teta.append([-(couleur+cote*1000),-(couleur+(sommet+1)*1000)])

            #print(phi)
            #teta.append(phi)

        print(teta)
            #     teta[sommet]=[[1,2],[-1,-2]]
            #     teta[sommet]=[[1,2,3],[-1,-2],[-1,-3],[-2,-3]]
            #       teta[sommet]=[[1,2,3,4],[-1,-2],[-1,-3],[-2,-3],[-1,-4],[-4,-3],[-4,-3]]


        #print(adj)


        self.base = teta  # a maj
        self.nbVariables = x             # a maj


    def execSolver(self) :
        """
        methode d'appel du solver sur la base de clauses representant le pb traite

        :return True si la base de clauses representant le probleme est satisfiable,
                False sinon
        """
        return SolverSAT.solve(self.base)

    def affBase(self) :
        """
        affichage de la base de clauses representant le probleme
        """
        print('Base de clause utilise ',self.nbVariables,' variables et contient les clauses suivantes : ')
        for cl in self.base :
            print(cl)


class __testEtape1__ :
    """
    methode principale de test
    """
    # TESTS
    # //////////////////////////////////////////////
    if __name__ == '__main__':

        # TEST 1 : town10.txt avec 3 couleurs
        print("Test sur fichier town10.txt avec 3 couleurs") ;
        e = Etape1("Data/town10.txt",True) ;
        e.majBase(3) ;
        e.affBase() ;
        print("Resultat obtenu (on attend True) :",e.execSolver()) ;


        # TEST 2 : town10.txt avec 2 couleurs
        print("Test sur fichier town10.txt avec 2 couleurs") ;
        e.majBase(2) ;
        e.affBase() ;
        print("Resultat obtenu (on attend False) :",e.execSolver()) ;


        # TEST 3 : town10.txt avec 4 couleurs
        print("Test sur fichier town10.txt avec 4 couleurs") ;
        e.majBase(4) ;
        e.affBase() ;
        print("Resultat obtenu (on attend True) :",e.execSolver()) ;


        # TEST 4 : flat20_3_0.col avec 4 couleurs
        print("Test sur fichier flat20_3_0.col avec 4 couleurs") ;
        e = Etape1("Data/pb-etape1/flat20_3_0.col",False) ;
        e.majBase(4) ;
        # e.affBase() ;
        print("Resultat obtenu (on attend True) :",e.execSolver()) ;

        # TEST 5 : flat20_3_0.col avec 3 couleurs
        print("Test sur fichier flat20_3_0.col avec 3 couleurs") ;
        e.majBase(3) ;
        # e.affBase() ;
        print("Resultat obtenu (on attend True) :",e.execSolver()) ;

        # TEST 6 : flat20_3_0.col avec 2 couleurs
        print("Test sur fichier flat20_3_0.col avec 2 couleurs") ;
        e.majBase(2) ;
        # e.affBase() ;
        print("Resultat obtenu (on attend False) :",e.execSolver()) ;



        # TEST 7 : jean.col avec 10 couleurs
        print("Test sur fichier jean.col avec 10 couleurs") ;
        e = Etape1("Data/pb-etape1/jean.col",False) ;
        e.majBase(10) ;
        # e.affBase() ;
        print("Resultat obtenu (on attend True) :",e.execSolver()) ;

        # TEST 9 : jean.col avec 9 couleurs
        print("Test sur fichier jean.col avec 9 couleurs") ;
        e.majBase(9) ;
        # e.affBase() ;
        print("Resultat obtenu (on attend False) :",e.execSolver()) ;

        # TEST 8 : jean.col avec 3 couleurs
        print("Test sur fichier jean.col avec 3 couleurs") ;
        e.majBase(3) ;
        # e.affBase() ;
        print("Resultat obtenu (on attend False) :",e.execSolver()) ;



Test sur fichier town10.txt avec 3 couleurs
10
[[-1001, -1002], [-1001, -1003], [-1002, -1003], [1001, 1002, 1003], [-1001, -2001], [-1002, -2002], [-1003, -2003], [-2001, -2001], [-2002, -2002], [-2003, -2003], [-2001, -2002], [-2001, -2003], [-2002, -2003], [2001, 2002, 2003], [-1, -3001], [-2, -3002], [-3, -3003], [-3001, -3001], [-3002, -3002], [-3003, -3003], [-4001, -3001], [-4002, -3002], [-4003, -3003], [-3001, -3002], [-3001, -3003], [-3002, -3003], [3001, 3002, 3003], [-1, -4001], [-2, -4002], [-3, -4003], [-3001, -4001], [-3002, -4002], [-3003, -4003], [-5001, -4001], [-5002, -4002], [-5003, -4003], [-4001, -4002], [-4001, -4003], [-4002, -4003], [4001, 4002, 4003], [-1001, -5001], [-1002, -5002], [-1003, -5003], [-2001, -5001], [-2002, -5002], [-2003, -5003], [-4001, -5001], [-4002, -5002], [-4003, -5003], [-6001, -5001], [-6002, -5002], [-6003, -5003], [-5001, -5002], [-5001, -5003], [-5002, -5003], [5001, 5002, 5003], [-1001, -6001], [-1002, -6002], [-1003, -6003], [-3001

# Etape 2

**Etape 2 = TP 3 à 6** Réalisation de la tâche 2.

Trois cas sont prévus :



*   **Cas 1 :** R2D2 commence par calculer le plus court chemin entre deux lieux de son monde. Pour cela, il utilise les distances entre les lieux, ainsi que les coordonnées cartésiennes de chaque lieu.
*   **Cas 2 :** Puis il cherche le chemin le plus court permettant de passer dans chaque lieu une seule fois puis de revenir ensuite à son point de départ.
*   **Cas 3 :** R2D2 vient d’être “upgradé” : son concepteur l’a équipé de la capacité à voler. Il peut désormais relier en ligne droite chacun des lieux de son monde sans être obligé de suivre les routes. Il peut donc trouver un autre chemin plus court permettant de passer par chaque lieu et de revenir ensuite à son point de départ.

Dans les trois cas, vous devrez identifier le nom et le type du problème, proposer un mode de représentation et le résoudre en utilisant un des algorithmes fournis. Pour cela vous compléterez les classes *EtatCas1*, *EtatCas2* et *EtatCas3* du package *etape2* et vous les utiliserez pour compléter et exécuter la classe *Etape2* du package *etape2*. Il peut être souhaitable de compléter les tests déjà proposés.

In [ ]:
"""
module pour l'état de l'etape 2 ds le cas 1
"""
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.etape2.Etat import Etat


class EtatCas1(Etat) :
    """ Classe pour definir un etat pour le cas 1 de la tache 2 (hérite de Etat)
    """

    # attributs
    # A COMPLETER
    # //////////////////////////////////////////////
    tg : GrapheDeLieux
    """ le graphe representant le monde """

    # constructeurs
    # A ECRIRE/MODIFIER/COMPLETER
    # //////////////////////////////////////////////
    def __init__(self, tg : GrapheDeLieux, param1 = None, param2 = None) :
        """ constructeur d'un etat a partir du graphe representant le monde

        :param tg: graphe representant le monde

        :param param1: a definir eventuellement

        :param param2: a definir eventuellement
        """
        self.tg = tg
        # a completer pour tenir compte de la presence ou pas des deux derniers parametres


    # methodes issues de Etat
    # //////////////////////////////////////////////
    def estSolution(self) :
        """ methode detectant si l'etat est une solution

        :return true si l'etat courant est une solution, false sinon
        """
        # A ECRIRE et MODIFIER le return en consequence
        return false


    def successeurs(self) :
        """ methode permettant de recuperer la liste des etats successeurs de l'etat courant

        :return liste des etats successeurs de l'etat courant
        """
        # A ECRIRE et MODIFIER le return en consequence
        return []


    def h(self) :
        """ methode permettant de recuperer l'heuristique de l'etat courant

        :return heuristique de l'etat courant
        """
        # A ECRIRE et MODIFIER le return en consequence
        return 0


    def k(self, e) :
        """ methode permettant de recuperer le cout du passage de l'etat courant à l'etat e

        :param e: un etat

        :return cout du passage de l'etat courant à l'etat e
        """
        # A ECRIRE et MODIFIER le return en consequence
        return 0


    def displayPath(self, pere) :
        """ methode pour afficher le chemin qui a mene a l'etat courant en utilisant la map des peres

        :param pere: map donnant pour chaque etat, son pere
        """
        # A ECRIRE
        print("resultat trouve : ")



    # methodes pour pouvoir utiliser cet objet dans des listes et des map
    # //////////////////////////////////////////////
    def __hash__(self) :
        """ methode permettant de recuperer le code de hachage de l'etat courant
        pour une utilisation dans des tables de hachage

        :return code de hachage
        """
        # A ECRIRE et MODIFIER le return en consequence
        return 0


    def __eq__(self, o) :
        """ methode de comparaison de l'etat courant avec l'objet o

        :param o: l'objet avec lequel on compare

        :return true si l'etat courant et o sont egaux, false sinon
        """
        # A ECRIRE et MODIFIER le return en consequence
        return False



    # methode pour affichage futur (heritee d'Object)
    # //////////////////////////////////////////////
    def __str__(self) :
        """ methode mettant l'etat courant sous la forme d'une
        chaine de caracteres en prevision d'un futur affichage

        :return representation de l'etat courant sour la forme d'une
        chaine de caracteres
        """
        # A ECRIRE et MODIFIER le return en consequence
        return ""





In [ ]:
"""
module pour l'état de l'etape 2 ds le cas 2
"""
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.etape2.Etat import Etat


class EtatCas2(Etat) :
    """
    Classe pour definir un etat pour le cas 2 de la tache 2
    """

    # attributs
    # A COMPLETER
    # //////////////////////////////////////////////
    tg : GrapheDeLieux
    """ le graphe representant le monde """

    # constructeurs
    # A ECRIRE/MODIFIER/COMPLETER
    # //////////////////////////////////////////////
    def __init__(self, tg : GrapheDeLieux, param1 = None, param2 = None) :
        """ constructeur d'un etat a partir du graphe representant le monde

        :param tg: graphe representant le monde

        :param param1: a definir eventuellement

        :param param2: a definir eventuellement
        """
        self.tg = tg
        # a completer pour tenir compte de la presence ou pas des deux derniers parametres


    # methodes issues de Etat
    # //////////////////////////////////////////////
    def estSolution(self) :
        """ methode detectant si l'etat est une solution

        :return true si l'etat courant est une solution, false sinon
        """
        # A ECRIRE et MODIFIER le return en consequence
        return false


    def successeurs(self) :
        """ methode permettant de recuperer la liste des etats successeurs de l'etat courant

        :return liste des etats successeurs de l'etat courant
        """
        # A ECRIRE et MODIFIER le return en consequence
        return []


    def h(self) :
        """ methode permettant de recuperer l'heuristique de l'etat courant

        :return heuristique de l'etat courant
        """
        # A ECRIRE et MODIFIER le return en consequence
        return 0


    def k(self, e) :
        """ methode permettant de recuperer le cout du passage de l'etat courant à l'etat e

        :param e: un etat

        :return cout du passage de l'etat courant à l'etat e
        """
        # A ECRIRE et MODIFIER le return en consequence
        return 0


    def displayPath(self, pere) :
        """ methode pour afficher le chemin qui a mene a l'etat courant en utilisant la map des peres

        :param pere: map donnant pour chaque etat, son pere
        """
        # A ECRIRE
        print("resultat trouve : ")



    # methodes pour pouvoir utiliser cet objet dans des listes et des map
    # //////////////////////////////////////////////
    def __hash__(self) :
        """ methode permettant de recuperer le code de hachage de l'etat courant
        pour une utilisation dans des tables de hachage

        :return code de hachage
        """
        # A ECRIRE et MODIFIER le return en consequence
        return 0


    def __eq__(self, o) :
        """ methode de comparaison de l'etat courant avec l'objet o

        :param o: l'objet avec lequel on compare

        :return true si l'etat courant et o sont egaux, false sinon
        """
        # A ECRIRE et MODIFIER le return en consequence
        return False



    # methode pour affichage futur (heritee d'Object)
    # //////////////////////////////////////////////
    def __str__(self) :
        """ methode mettant l'etat courant sous la forme d'une
        chaine de caracteres en prevision d'un futur affichage

        :return representation de l'etat courant sour la forme d'une
        chaine de caracteres
        """
        # A ECRIRE et MODIFIER le return en consequence
        return ""




In [ ]:
"""
module pour l'état de l'etape 2 ds le cas 3
"""
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.etape2.Etat import Etat

class EtatCas3(Etat) :
    """
    Classe pour definir un etat pour le cas 3 de la tache 2
    """
    # attributs
    # A COMPLETER
    # //////////////////////////////////////////////
    tg : GrapheDeLieux
    """ le graphe representant le monde """

    # constructeurs
    # A ECRIRE/MODIFIER/COMPLETER
    # //////////////////////////////////////////////
    def __init__(self, tg : GrapheDeLieux, param1 = None, param2 = None) :
        """ constructeur d'un etat a partir du graphe representant le monde

        :param tg: graphe representant le monde

        :param param1: a definir eventuellement

        :param param2: a definir eventuellement
        """
        self.tg = tg
        # a completer pour tenir compte de la presence ou pas des deux derniers parametres


    # methodes issues de Etat
    # //////////////////////////////////////////////
    def estSolution(self) :
        """ methode detectant si l'etat est une solution

        :return true si l'etat courant est une solution, false sinon
        """
        # A ECRIRE et MODIFIER le return en consequence
        return false


    def successeurs(self) :
        """ methode permettant de recuperer la liste des etats successeurs de l'etat courant

        :return liste des etats successeurs de l'etat courant
        """
        # A ECRIRE et MODIFIER le return en consequence
        return []


    def h(self) :
        """ methode permettant de recuperer l'heuristique de l'etat courant

        :return heuristique de l'etat courant
        """
        # A ECRIRE et MODIFIER le return en consequence
        return 0


    def k(self, e) :
        """ methode permettant de recuperer le cout du passage de l'etat courant à l'etat e

        :param e: un etat

        :return cout du passage de l'etat courant à l'etat e
        """
        # A ECRIRE et MODIFIER le return en consequence
        return 0


    def displayPath(self, pere) :
        """ methode pour afficher le chemin qui a mene a l'etat courant en utilisant la map des peres

        :param pere: map donnant pour chaque etat, son pere
        """
        # A ECRIRE
        print("resultat trouve : ")



    # methodes pour pouvoir utiliser cet objet dans des listes et des map
    # //////////////////////////////////////////////
    def __hash__(self) :
        """ methode permettant de recuperer le code de hachage de l'etat courant
        pour une utilisation dans des tables de hachage

        :return code de hachage
        """
        # A ECRIRE et MODIFIER le return en consequence
        return 0


    def __eq__(self, o) :
        """ methode de comparaison de l'etat courant avec l'objet o

        :param o: l'objet avec lequel on compare

        :return true si l'etat courant et o sont egaux, false sinon
        """
        # A ECRIRE et MODIFIER le return en consequence
        return False



    # methode pour affichage futur (heritee d'Object)
    # //////////////////////////////////////////////
    def __str__(self) :
        """ methode mettant l'etat courant sous la forme d'une
        chaine de caracteres en prevision d'un futur affichage

        :return representation de l'etat courant sour la forme d'une
        chaine de caracteres
        """
        # A ECRIRE et MODIFIER le return en consequence
        return ""





In [ ]:
"""
module principal pour l'etape 2
"""

from projet.outils.GrapheDeLieux import GrapheDeLieux
# rajouter ensuite le import permettant d'utiliser le solver (l'algo) choisi
# from solvers.... import ...


class Etape2 :
    """
    classe pour realiser les tests de l'etape 2 du projet (execution des cas 1, 2 et 3 : tache 2)
    """


    # tests
    # ////////////////////////////////////////////
    """ methode de TESTS pour Etape2
    """
    if __name__ == '__main__':

        # ///////////////////////////////////////////////
        # tests sur CAS 1 : entre 2 villes donnees
        # ///////////////////////////////////////
        # cas : 10 villes de 0 à 9
        tg = GrapheDeLieux.loadGraph("Data/town10.txt",True)
        cas1 = EtatCas1(tg)
        print("======== TEST CAS 1 10 villes de 0 a 9 : \n")
        # choisir ici un algo et l'executer

        # cas : 10 villes de 5 à 9
        cas1 = EtatCas1(tg,5,9)
        print("======== TEST CAS 1 10 villes de 5 a 9 : \n")
        # choisir ici un algo et l'executer

        # cas : 10 villes de 2 à 9
        cas1 = EtatCas1(tg,2,9)
        print("======== TEST CAS 1 10 villes de 2 a 9 : \n")
        # choisir ici un algo et l'executer

        # cas : 10 villes de 1 à 7
        cas1 = EtatCas1(tg,1,7)
        print("======== TEST CAS 1 10 villes de 1 a 7 : \n")
        # choisir ici un algo et l'executer

        # cas : 26 villes de 0 à 25
        tg = GrapheDeLieux.loadGraph("Data/town30.txt",True)
        cas1 = EtatCas1(tg)
        print("======== TEST CAS 1 26 villes de 0 a 25 : \n")
        # choisir ici un algo et l'executer

        # cas : 146 villes de 0 à 145
        tg = GrapheDeLieux.loadGraph("Data/town150.txt",True)
        cas1 = EtatCas1(tg)
        print("======== TEST CAS 1 146 villes de 0 a 145 : \n")
        # choisir ici un algo et l'executer

        # cas : 998 villes de 0 à 997
        tg = GrapheDeLieux.loadGraph("Data/town1000.txt",True)
        cas1 = EtatCas1(tg)
        print("======== TEST CAS 1 1000 villes : \n")
        # choisir ici un algo et l'executer


        # ///////////////////////////////////////////////
        # tests sur CAS 2 : tour complet par voie de terre
        # ///////////////////////////////////////////
        # cas : 10 villes de 0 à 9
        tg = GrapheDeLieux.loadGraph("Data/town10.txt",True)
        cas2 = EtatCas2(tg)
        print("======== TEST CAS 2 10 villes de 0 a 9 : \n")
        # choisir ici un algo et l'executer


        # ///////////////////////////////////////////////
        # tests sur CAS 3 : tour complet par voie des airs
        # ///////////////////////////////////////////////
        # cas : 6 villes de 0 à 5
        tg = GrapheDeLieux.loadGraph("Data/town6.txt",True)
        cas3 = EtatCas3(tg)
        print("======== TEST CAS 3 6 villes de 0 a 5 : \n")
        # choisir ici un algo et l'executer

        # cas : 7 villes de 0 à 6
        tg = GrapheDeLieux.loadGraph("Data/town7.txt",True)
        cas3 = EtatCas3(tg)
        print("======== TEST CAS 3 7 villes de 0 a 6 : \n")
        # choisir ici un algo et l'executer

        # cas : 8 villes de 0 à 7
        tg = GrapheDeLieux.loadGraph("Data/town8.txt",True)
        cas3 = EtatCas3(tg)
        print("======== TEST CAS 3 8 villes de 0 a 7 : \n")
        # choisir ici un algo et l'executer

        # cas : 9 villes de 0 à 8
        tg = GrapheDeLieux.loadGraph("Data/town9.txt",True)
        cas3 = EtatCas3(tg)
        print("======== TEST CAS 3 9 villes de 0 a 8 : \n")
        # choisir ici un algo et l'executer

        # cas : 10 villes de 0 à 9
        tg = GrapheDeLieux.loadGraph("Data/town10.txt",True)
        cas3 = EtatCas3(tg)
        print("======== TEST CAS 3 10 villes de 0 a 9 : \n")
        # choisir ici un algo et l'executer

        # cas : 11 villes de 0 à 10
        tg = GrapheDeLieux.loadGraph("Data/town11.txt",True)
        cas3 = EtatCas3(tg)
        print("======== TEST CAS 3 11 villes de 0 a 10 : \n")
         # choisir ici un algo et l'executer



10
======== TEST CAS 1 10 villes de 0 a 9 : 

======== TEST CAS 1 10 villes de 5 a 9 : 

======== TEST CAS 1 10 villes de 2 a 9 : 

======== TEST CAS 1 10 villes de 1 a 7 : 

26
======== TEST CAS 1 26 villes de 0 a 25 : 

146
======== TEST CAS 1 146 villes de 0 a 145 : 

998
======== TEST CAS 1 1000 villes : 

10
======== TEST CAS 2 10 villes de 0 a 9 : 

6
======== TEST CAS 3 6 villes de 0 a 5 : 

7
======== TEST CAS 3 7 villes de 0 a 6 : 

8
======== TEST CAS 3 8 villes de 0 a 7 : 

9
======== TEST CAS 3 9 villes de 0 a 8 : 

10
======== TEST CAS 3 10 villes de 0 a 9 : 

11
======== TEST CAS 3 11 villes de 0 a 10 : 



# Etape 3

**Etape 3 = TP 7 et 8** Réalisation de la tâche 2 (suite).

R2D2 se rend compte que sa méthode précédente met trop de temps à s’exécuter ! Du coup, il renonce à trouver le chemin le plus court et est prêt à tenter des chemins un peu moins bons pourvu qu’il arrive à les calculer plus vite. Et comme il est curieux, il va essayer deux méthodes différentes pour voir celle
qui est la plus efficace. Vous devrez proposer un mode de représentation et résoudre le problème en utilisant au moins deux des algorithmes fournis.
Pour cela vous compléterez la classe *UneSolution* du package *etape3* et vous l’utiliserez pour compléter et exécuter la classe *Etape3* du package *etape3*. Il peut être souhaitable de compléter les tests déjà proposés.

In [ ]:
"""  module pour la classe UneSolution """

from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.etape3.Solution import Solution

class UneSolution(Solution) :
    """
    Classe pour definir une solution pour le cas 3 de la tache 2 (hérite de Solution)
    """


    #    attributs
    #    A COMPLETER
    #    //////////////////////////////////////////////
    tg : GrapheDeLieux
    """  le graphe representant le monde """

    #    constructeurs
    #    A ECRIRE/COMPLETER
    #    //////////////////////////////////////////////
    def __init__(self, tg : GrapheDeLieux) :
        """  constructeur d'une solution a partir du graphe representant le monde

        :param tg: graphe representant le monde
        """
        #    A ECRIRE en completant eventuellement par des parametres optionnels
        self.tg = tg



    #    methodes de la classe abstraite Solution
    #    //////////////////////////////////////////////
    def lesVoisins(self) :
        """  methode recuperant la liste des voisins de la solution courante

        :return liste des voisins de la solution courante
        """
        #    A ECRIRE et MAJ la valeur retournee
        return None


    def unVoisin(self) :
        """  methode recuperant un voisin de la solution courante

        :return voisin de la solution courante
        """
        #    A ECRIRE et MAJ la valeur retournee
        return None


    def eval(self) :
        """  methode recuperant la valeur de la solution courante

        :return valeur de la solution courante
        """
        #    A ECRIRE et MAJ la valeur retournee
        return 0


    def nelleSolution(self) :
        """  methode generant aleatoirement une nouvelle solution
        a partir de la solution courante

        :return nouvelle solution generee aleatoirement a partir de la solution courante
        """
        #    A ECRIRE et MAJ la valeur retournee
        return None


    def displayPath(self) :
        """  methode affichant la solution courante comme un chemin dans le graphe
        """
        #    A ECRIRE
        print("la meilleure solution est :")


    #    methodes pour pouvoir utiliser cet objet dans des listes et des map
    #    //////////////////////////////////////////////
    def __hash__(self) :
        """  methode permettant de recuperer le code de hachage de la solution courante
        pour une utilisation dans des tables de hachage

        :return code de hachage
        """
        #    A ECRIRE et MODIFIER le return en consequence
        return 0


    def __eq__(self,o) :
        """  methode de comparaison de la solution courante avec l'objet o

        :param o: l'objet avec lequel on compare

        :return True si la solution courante et o sont egaux, False sinon
        """
        #    A ECRIRE et MODIFIER le return en consequence
        return False



    #    methode pour affichage futur (heritee d'Object)
    #    //////////////////////////////////////////////
    def __str__(self) :
        """  methode mettant la solution courante sous la forme d'une
        chaine de caracteres en prevision d'un futur affichage

        :return representation de la solution courante sour la forme d'une chaine de caracteres
        """
        #    A ECRIRE et MODIFIER le return en consequence
        return ""




In [ ]:
"""
module principal pour l'etape 3
"""

from projet.outils.GrapheDeLieux import GrapheDeLieux

# rajouter ensuite le import permettant d'utiliser les solvers choisis
# from projet.solvers.... import ...


class Etape3 :
    """  classe pour realiser les tests de l'etape 3 du projet (suite tache 2) """


    #    tests
    #    ////////////////////////////////////////////
    """  methode de TESTS pour Etape3
    """
    if __name__ == '__main__':


        #    tests sur Etape 3
        #    ///////////////////
        #    cas 1 : 10 villes de 0 à 9
        tg : GrapheDeLieux = GrapheDeLieux.loadGraph("Data/town10.txt",True)
        tsp : UneSolution = UneSolution(tg)
        print("======== Solver 1 pour 10 villes de 0 a 9 : \n")
        #    appel du solver 1
        #    ...
        print("======== Solver 2 pour 10 villes de 0 a 9 : \n")
        #    appel du solver 2
        #    ...

        #    ///////////////////
        #    cas 2 : 26 villes de 0 à 25
        tg = GrapheDeLieux.loadGraph("Data/town30.txt",True)
        tsp = UneSolution(tg)
        print("======== Solver 1 pour 26 villes de 0 a 25 : \n")
        #    appel du solver 1
        #    ...
        print("======== Solver 2 pour 26 villes de 0 a 25 : \n")
        #    appel du solver 2
        #    ...


        #    ///////////////////
        #    cas 3 : 150 villes
        tg = GrapheDeLieux.loadGraph("Data/town150.txt",True)
        tsp = UneSolution(tg)
        print("======== Solver 1 pour 150 villes : \n")
        #    appel du solver 1
        #    ...
        print("======== Solver 2 pour 150 villes : \n")
        #    appel du solver 2
        #    ...


        #    ///////////////////
        #    cas 4 : 1000 villes
        tg = GrapheDeLieux.loadGraph("Data/town1000.txt",True)
        tsp = UneSolution(tg)
        print("======== Solver 1 pour 1000 villes : \n")
        #    appel du solver 1
        #    ...
        print("======== Solver 2 pour 1000 villes : \n")
        #    appel du solver 2
        #    ...



# Etape 4

**Etape 4 = TP 9 et 10** Réalisation de la tâche 3.

Vous devrez identifier le nom et le type du problème, proposer un encodage sous la forme d’un graphe de contraintes et le résoudre en utilisant un des algorithmes fournis. Pour cela vous compléterez et exécuterez la classe *Etape4* du package *etape4*. Il peut être souhaitable de compléter les tests déjà proposés.

In [ ]:
"""
module principal pour l'etape 4
"""

from projet.outils.GrapheDeLieux import GrapheDeLieux

# rajouter ensuite le import permettant d'utiliser le solver (l'algo) choisi
# from projet.solvers.... import ...

class Etape4 :
    """
    classe de test pour l'etape 4
    """

    if __name__ == '__main__':

        print("\n========== TEST  ==========")
        print("=============================")

        #    TEST 1 : town10.txt avec 3 couleurs
        tg : GrapheDeLieux = GrapheDeLieux.loadGraph("Data/town10.txt",True)
        print("\nTest sur town10 avec 3 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer

        #    TEST 2 : town10.txt avec 2 couleurs
        print("\nTest sur town10 avec 2 couleurs (on attend NOK) :")
        #    choisir ici un algo et l'executer

        #    TEST 3 : town10.txt avec 4 couleurs
        print("\nTest sur town10 avec 4 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer



        #    TEST 4 : flat20_3_0.col avec 4 couleurs
        tg = GrapheDeLieux.loadGraph("Data/pb-etape1/flat20_3_0.col",False)
        print("Test sur flat20_3_0.col avec 4 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer

        #    TEST 5 : flat20_3_0.col avec 3 couleurs
        print("Test sur flat20_3_0.col avec 3 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer

        #    TEST 6 : flat20_3_0.col avec 2 couleurs
        print("Test sur flat20_3_0.col avec 2 couleurs (on attend NOK) :")
        #    choisir ici un algo et l'executer



        #    TEST 7 : jean.col avec 10 couleurs
        tg = GrapheDeLieux.loadGraph("Data/pb-etape1/jean.col",False)
        print("Test sur jean.col avec 10 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer

        #    TEST 9 : jean.col avec 3 couleurs
        print("Test sur jean.col avec 3 couleurs (on attend NOK) :")
        #    choisir ici un algo et l'executer

        #    TEST 8 : jean.col avec 9 couleurs
        print("Test sur jean.col avec 9 couleurs (on attend NOK) :")
        #    choisir ici un algo et l'executer






# Etape 5

**Etape 5 = TP 11 et 12** Réalisation de la tâche 2 (suite et fin).

Comme R2D2 aime aussi beaucoup les maths et qu’il veut épater ses concepteurs, il reprend le cas 3 de la tâche 2, et cherche à le résoudre en l’exprimant à l’aide de formules mathématiques. Vous devrez proposer un encodage approprié utilisant le langage ZIMPL et résoudre le problème en utilisant le solver SCIP.

**Installation de l'environnement de travail**


*   récupération de l'outil SCIP sur votre machine (voir le README dans la documentation sur ZIMPL et sur SCIP qui se trouve dans le fichier *Doc.zip* que vous pouvez télécharger depuis la liste des fichiers et installer en le décompressant sur votre machine.)
*   compression du répertoire SCIP sur votre machine (celui qui contient les 3 sous-répertoire "bin", "lib" et "include") pour créer le fichier *SCIP.zip*
*   puis avec le bloc de code suivant : upload du fichier *SCIP.zip* sur GoogleCoLab, décompression et test avec le code ZIMPL donné





In [ ]:
from google.colab import files

files.upload()

!unzip SCIP.zip

!SCIP/bin/scip -f projet/etape5/test.zpl



**Exemple de code ZIMPL utilisé pour le test précédent :** il correspond à l'exemple vu en cours (résultat attendu : x1 = 4, x2 = 0 pour une valeur de 68)


> var x1 integer >= 0 ;

> var x2 integer >= 0 ;

> maximize res : (17 * x1) + (12 * x2) ;

> subto c1 : (10 * x1) + (7 * x2) <= 40 ;

> subto c2 : x1 + x2 <= 5 ;




**Le travail à faire**

Ecrire un programme ZIMPL répondant à la question posée. Ce programme sera à placer et à tester sur le modèle du bloc de code suivant.

Si vous définissez plusieurs programmes ZIMPL, vous devrez faire un bloc de code par programme. Attention à bien choisir le nom des fichiers de sauvegarde pour ne pas en écraser.


In [ ]:
from google.colab import files

contenuFichierZIMPL = """
    var x1 integer >= 0 ;
    var x2 integer >= 0 ;
    maximize res : (17 * x1) + (12 * x2) ;
    subto c1 : (10 * x1) + (7 * x2) <= 40 ;
    subto c2 : x1 + x2 <= 5 ;
    """

with open('projet/etape5/fichier1.zpl', 'w') as f:
  f.write(contenuFichierZIMPL)

!SCIP/bin/scip -f projet/etape5/fichier1.zpl

